In [1]:
# import the necessary packages
from musicsearchmodel.model import SiameseModel
from matplotlib import pyplot as plt
from musicsearchmodel import config
from sklearn.metrics import confusion_matrix
from tensorflow import keras
import tensorflow as tf
import numpy as np
import os

In [2]:
### define function to compute metrics
def metrics(predictions, label_gt):
    TP_binary = np.logical_and(predictions, label_gt)
    FP_binary = np.logical_and(predictions, np.logical_not(label_gt))
    TN_binary = np.logical_and(np.logical_not(predictions), np.logical_not(label_gt))
    FN_binary = np.logical_and(np.logical_not(predictions), label_gt)
    
    TP = sum(TP_binary )
    FP = sum(FP_binary )
    TN = sum(TN_binary )
    FN = sum(FN_binary )
    precision = TP/(TP+FP)
    recall = TP/(FN+TP)
    F1 = (2*precision*recall)/(precision+recall)
    
    return precision, recall, F1

In [3]:
modelPath = config.MODEL_PATH
### Loading pre-trained siamese model for inference
print(f"[INFO] loading the siamese network from {modelPath}...")
siameseNetwork = keras.models.load_model(filepath=modelPath)
siameseModel = SiameseModel(
	siameseNetwork=siameseNetwork,
	margin=0.5,
	lossTracker=keras.metrics.Mean(name="loss"),
)

[INFO] loading the siamese network from output\siamese_network...


In [ ]:
musicDatabasePath = ''
img_height, img_width = config.IMAGE_SIZE
print(f"[INFO] Setting-up Data Pipeline...")
test_ds = tf.keras.utils.image_dataset_from_directory(
  config.TEST_DATASET,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=1)
music_ds = tf.keras.utils.image_dataset_from_directory(
  musicDatabasePath,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=1)

In [ ]:
songs = []
songLabels = []
for entry in music_ds:
    song, songLabel = entry
    song_spectrogram = song/255
    songs.append(song_spectrogram)
    songLabels.append(songLabel)

In [ ]:
print(f"[INFO] Making Predictions on Test Set...")
predictions = []
labels = []
for batch in test_ds:
    batch_img, label = batch
    image = batch_img/255

In [ ]:
pred_distances = []

In [ ]:
for anchor in songs:
    (apDistance, anDistance) = siameseModel((image , anchor, image))
    pred_distances.append(apDistance.numpy())

In [ ]:
predictions.append(songLabels[np.argmin(pred_distances)][0])  
labels.append(label.numpy()[0])

In [ ]:
labels = np.asarray(labels)
predictions = np.asarray(predictions)
print(f"[INFO] Evaluating the model...")
##Computing Metrics
N = labels.shape[0]
accuracy = (labels == predictions).sum() / N

In [ ]:
for pos_label in [0,1,2,3,4]:
    pred = (predictions == pos_label)
    label_gt = (labels == pos_label)
    precision, recall, F1_score = metrics(pred,label)

In [ ]:
cm = confusion_matrix(labels, predictions)
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)

In [ ]:
recallOverall = np.mean(recall)
precisionOverall = np.mean(precision)
F1_overall = 2*recallOverall*precisionOverall/(recallOverall+precisionOverall)